# CARTE-Enbridge Bootcamp
#### Lab 2-1

# Introduction to TensorFlow

In this lab, we will go over the basics of using TensorFlow to build and train a neural network. TensorFlow is one of the two most popular deep learning frameworks (the other being PyTorch). It is developed by Google and is used in many of their products. 

Using the sub-module Keras, we will build a simple neural network to classify images of handwritten digits from the MNIST dataset.

First, we will import TensorFlow and check the version:

In [ ]:
# Import TensorFlow
import tensorflow as tf
import tensorflow.keras as keras

print("Using TensorFlow version", tf.__version__)

# Use GPU, if available
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
    print("GPU device not found")
else:
    print(f"Found GPU at: {device_name}")

Now we will download the MNIST dataset. Keras provides a convenient function for this. The dataset is already split into training and test sets. We will use the training set to train the model and the test set to evaluate the model's performance on unseen data.

In [ ]:
# Download and prepare the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Before we move forward, it's always helpful to visualize our data to get a sense of what we're working with. Let's display a few of the images from the training set along with their corresponding labels:

In [ ]:
# Visualize the first five images from the training dataset
import matplotlib.pyplot as plt

%matplotlib inline
plt.figure(figsize=(10, 10))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(y_train[i])
plt.show()

Let's establish some basic information about our dataset, while we're at it:

In [ ]:
input_shape = x_train[0].shape
num_classes = len(set(y_train))
print("There are", num_classes, "classes in our dataset")
print("The shape of each image is", input_shape)

When working with neural networks, it is important to normalize the data so that the values all fall between 0 and 1. This is done by dividing each value by the maximum value in the dataset, which is 255 in the case of the MNIST dataset. 

We will also one-hot encode the labels. One-hot encoding is a process where we replace each label with a vector of length equal to the number of possible classes. It's called 'one-hot' because only one of the values in the vector is 1 (aka 'hot'), and the rest are 0.

Part of the reason we do this here is that even though the labels are numbers, they are not ordinal. That is, the fact that the label for a 3 is greater than the label for a 2 does not mean that a 3 is more similar to a 2 than it is to a 4. In fact, the labels are categorical, not numerical. One-hot encoding allows us to treat the labels as categorical, which is important for the loss function we will use later on.
 
This means that we will convert the labels from a single number to a vector whose length is equal to the number of possible classes. The vector will be all 0s except for the index corresponding to the label, which will be 1. For example, if the label is 3, the one-hot encoded label will be [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].

In [ ]:
# Normalize the data so that the values all fall between 0 and 1
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

In [ ]:
# One-hot encode the labels
print(f"Before one-hot encoding: {y_train[0]}")
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print(f"After one-hot encoding:\n {y_train[0]}")

The last thing we will do before getting to building our actual model is reshape our input. The images in the MNIST dataset are 28x28 pixels, but we need to flatten them into a single vector of length 784 in order to feed them into our neural network. Later on in the lab, we'll look at a neural network that can accept a 2D image, but for now we will keep it simple. We will do this using the reshape() function.

In [ ]:
# Flatten the input images
x_train_flattened = x_train.reshape(60000, 28 * 28)
x_test_flattened = x_test.reshape(10000, 28 * 28)

fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(x_train[0], cmap=plt.cm.binary)
ax[0].set_title("Before flattening")
ax[1].bar(range(28 * 28), x_train_flattened[0])
ax[1].set_title("After flattening")
plt.show()

Okay, now we're ready to build our model! The Keras Sequential API makes this very easy. We just define one layer of our neural network at a time, starting with the input layer. The first layer we add must specify the input shape, which is (28*28,) in our case. 

We will also be using two activation functions we haven't seen before - inside the network, we will use ReLU, and at the end we will use Softmax. ReLU stands for Rectified Linear Unit, and it is defined as f(x) = max(0, x). It is a very simple function, but it is very effective in neural networks. 

Softmax is a function that takes a vector of numbers and outputs a vector of the same length, where each value is between 0 and 1 and the sum of the values is 1. It is often used as the activation function for the output layer of a neural network, because it allows us to interpret the output as a probability distribution over the possible classes.

In [ ]:
import numpy as np

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
# Plot the ReLU function
x = np.linspace(-5, 5, 100)
ax[0].plot(x, np.maximum(0, x))
ax[0].set_title("ReLU")
ax[0].set_xlabel("x")
ax[0].set_ylabel("f(x)")
# Plot the softmax function
ax[1].plot(x, np.exp(x) / np.sum(np.exp(x)))
ax[1].set_title("Softmax")
ax[1].set_xlabel("x")
ax[1].set_ylabel("f(x)")
plt.show()

In [ ]:
# First, define the input to the neural network
input = keras.layers.Input(shape=(28 * 28,)) # Aka (784,)

In [ ]:
# Next, define the first hidden layer
hidden1 = keras.layers.Dense(128, activation="relu")(input) # 128 neurons, ReLU activation

In [ ]:
# Next, define the second hidden layer
hidden2 = keras.layers.Dense(64, activation="relu")(hidden1) # 64 neurons, ReLU activation

In [ ]:
# Finally, define the output layer
output = keras.layers.Dense(10, activation="softmax")(hidden2) # 10 neurons, softmax activation

In [ ]:
# Now we can define the model, specifying the input and output layers
model = keras.models.Model(inputs=input, outputs=output)
model.summary()

In [ ]:
# Visualize the model
keras.utils.plot_model(model, show_shapes=True)

It really is that easy! Now we just need to compile the model, specifying the optimizer, loss function, and metrics we want to use. We will use the standard stochastic gradient descent optimizer, the categorical cross-entropy loss function, and the accuracy metric. We use categorical cross-entropy because we have more than two classes. If we had only two classes, we would use binary cross-entropy.

In [ ]:
optimizer = "sgd" # Stochastic gradient descent - the foundation of most neural network optimizers
loss = "categorical_crossentropy" # The loss function we will use to train the model - categorical cross-entropy for more than two classes
metrics = ["accuracy"] # The metric we will use to evaluate the model - accuracy is the proportion of correct predictions

In [ ]:
# Compile the model
model.compile(
    optimizer=optimizer, 
    loss=loss, 
    metrics=metrics
)

Now we can train the model. We just need to specify the training data, the number of epochs, and the batch size. The batch size is the number of training examples that are fed into the model at once. The number of epochs is the number of times the model will see the entire training set.

In [ ]:
# Train the model
model.fit(x_train_flattened, y_train, epochs=5, batch_size=32, validation_split=0.1)

At the end of each epoch, the model will evaluate its performance on the validation set - this is a held out part of our training data that allows us to monitor how well the model is training. It gives us an idea of how well the model will generalize to unseen data. But we still need to evaluate the model on the test set to get a final measure of its performance:

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_flattened, y_test)
print(f"Test loss: {loss:.2f}")
print(f"Test accuracy: {accuracy*100:.2f}%")

That is very good! Let's take a look at some of the predictions the model made:

In [ ]:
# Visualize the first 5 correct predictions
import numpy as np

predictions = model.predict(x_test_flattened)
correct_indices = np.nonzero(
    np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)
)[0]
plt.figure(figsize=(10, 10))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[correct_indices[i]].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(np.argmax(predictions[correct_indices[i]]))
plt.show()

In [ ]:
# Visualize the first 5 incorrect predictions
incorrect_indices = np.nonzero(
    np.argmax(predictions, axis=1) != np.argmax(y_test, axis=1)
)[0]
plt.figure(figsize=(10, 10))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[incorrect_indices[i]].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(f'{np.argmax(predictions[incorrect_indices[i]])} (True label: {np.argmax(y_test[incorrect_indices[i]])})')
plt.show()

As above, it can be helpful to look at some of the incorrect predictions to get a sense of what the model is getting wrong. Sometimes, there might be something surprising that we can learn from, but here it seems like the model is just getting stuck on some of the more unusually written digits.

## Convolutional Neural Networks

Now we will build a convolutional neural network to classify the same images. Convolutional neural networks are a special type of neural network that are designed to work with images. With convolutional layers, a "filter" is learned, which is a small matrix of weights. The filter is applied to each part of the image, and the output is a new image that is a "filtered" version of the original image. The filters are learned during training, and the goal is for the filters to learn to detect certain features of the image, such as edges or corners.

In this way, CNNs can accept 2D images as input, rather than a flattened vector. This allows us to preserve the spatial information of the image, which is important for image classification.

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Build the model

input = keras.layers.Input(
    shape=(28, 28, 1)
)  # Instead of a flattened vector, we specify the shape of the input to be a 28x28 image with 1 channel (grayscale)

conv1 = Conv2D(32, (3, 3), activation="relu")(
    input
)  # Our first convolutional layer - uses a 3x3 filter of weights to produce 32 filtered images

conv2 = Conv2D(64, (3, 3), activation="relu")(
    conv1
)  # Our second convolutional layer - uses a 3x3 filter of weights to produce 64 filtered images

flatten = Flatten()(
    conv2
)  # Flatten the output of the convolutional layers into a vector, so we can make a prediction

hidden1 = Dense(128, activation="relu")(
    flatten
)  # Our first hidden layer - takes in the flattened output of the convolutional layers, and outputs 128 values

output = Dense(10, activation="softmax")(
    hidden1
)  # Our output layer - takes in the output of the first hidden layer, and outputs 10 values (one for each possible class)

Like before, we just need to compile the model, specifying the optimizer, loss function, and metrics we want to use. We will use the Adam optimizer, the categorical cross-entropy loss function, and the accuracy metric.

In [ ]:
model = keras.models.Model(inputs=input, outputs=output)
model.summary()

In [ ]:
# Compile the model - this time we will use adam, a more advanced optimizer based on stochastic gradient descent
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss:.2f}")
print(f"Test accuracy: {accuracy*100:.2f}%")

Between swapping out stochastic gradient descent for Adam, and using convolutional layers instead of dense layers, we were able to improve our accuracy by 1.5%!

Next, we are going to introduce a couple of 'Callbacks'. These are functions that are called during training. We will use two callbacks: EarlyStopping and ModelCheckpoint. 

EarlyStopping will stop the training process if the validation loss stops improving. This means that we don't have to specify the number of epochs ahead of time - the model will train until it stops improving, and then stop automatically.

ModelCheckpoint will save the best model (the one with the lowest validation loss) to a file. This is useful because we can then load the best model and evaluate it on the test set.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor="val_loss", patience=5
)  # Stop training if the validation loss hasn't improved in 5 epochs

model_checkpoint = ModelCheckpoint(
    "best_model", monitor="val_loss", save_best_only=True
)  # Save the model with the lowest validation loss to a folder called 'best_model'

This part is the same as before, but we want a new model so we'll run it again:

In [ ]:
input = keras.layers.Input(shape=(28, 28, 1))
conv1 = Conv2D(32, (3, 3), activation="relu")(input)
pool1 = MaxPooling2D((2, 2))(
    conv1
)  # Add a pooling layer to reduce the size of the output of the convolutional layer - this will speed up training
conv2 = Conv2D(64, (3, 3), activation="relu")(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)  # Add another pooling layer
flatten = Flatten()(pool2)
hidden1 = Dense(128, activation="relu")(flatten)
output = Dense(10, activation="softmax")(hidden1)
model = keras.models.Model(inputs=input, outputs=output)

And now we can train:

In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model with callbacks
history = model.fit(
    x_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping, model_checkpoint],
)

In [ ]:
# Load the best model
model = keras.models.load_model("best_model")

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss:.2f}")
print(f"Test accuracy: {accuracy*100:.2f}%")

When we train a model, we can also access the history of the training process. This includes the loss and accuracy on the training and validation sets at each epoch. We can use this to plot the training and validation loss over time, which can help us diagnose problems with our model.

In [ ]:
# Plot the training and validation loss
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["Train", "Validation"])
plt.show()

Let's take a look at some of the incorrect predictions for this model:

In [ ]:
# Visualize the first 5 incorrect predictions
predictions = model.predict(x_test)
incorrect_indices = np.nonzero(
    np.argmax(predictions, axis=1) != np.argmax(y_test, axis=1)
)[0]
plt.figure(figsize=(10, 10))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[incorrect_indices[i]], cmap=plt.cm.binary)
    plt.xlabel(
        f"{np.argmax(predictions[incorrect_indices[i]])} (True label: {np.argmax(y_test[incorrect_indices[i]])})"
    )
plt.show()

## Your Turn

Now it's your turn to build a neural network! We will use the CIFAR-10 dataset, which consists of 60,000 32x32 color images in 10 classes. We will build a neural network to classify these images.

First, we will download the dataset:

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

# Preview the first 5 images
plt.figure(figsize=(10, 10))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    plt.xlabel(class_names[y_train[i][0]])
plt.show()

In [ ]:
# Normalize the data
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

In [ ]:
# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

Using the example code above, build a model that can classify these images. You can copy directly one of the model definitions from above, but if you feel like it, you can also extend the network further. You can get more information about available layers and other options in the [Keras documentation](https://keras.io/).

In [ ]:
## Your code here